In [1]:
from collections import deque
import pickle

def select_root_with_minimum_m(G, k):
    m_values = {}
    
    for v in range(len(G)):
        m_v = 0
        while len(get_N_h(G, v, m_v)) < k:
            m_v += 1
        m_values[v] = m_v
    return min(m_values, key=m_values.get)

def children_of(G, v, visited):
    return [node for node in G[v] if node not in visited]

def get_N_h(G, v, h):
    visited = set()
    queue = deque([(v, 0)])

    while queue:
        node, depth = queue.popleft()
        if node not in visited:
            visited.add(node)
            if depth < h:
                for neighbor in G[node]:
                    if neighbor not in visited:
                        queue.append((neighbor, depth + 1))
    return visited

def r_m_values(G, labels_assigned, k):
    r_values = {}
    m_values = {}

    for v in range(len(G)):
        r_v = 0
        m_v = 0
        while len(set(labels_assigned[u] for u in get_N_h(G, v, r_v))) < k:
            r_v += 1
        r_values[v] = r_v

        while len(get_N_h(G, v, m_v)) < k:
            m_v += 1
        m_values[v] = m_v

    return r_values, m_values

def is_valid(v, color, labels_assigned, G, k):
    neighbors = G[v]
    labels_in_m = set()
    m_v = 0
    
    while len(labels_in_m) < k:
        labels_in_m = set(labels_assigned[u] for u in get_N_h(G, v, m_v) if u in labels_assigned)
        m_v += 1
    
    if color in labels_in_m:
        return False
    
    return True

def tree_node_labelling_util(G, k, labels_assigned, v):
    if v == len(G):
        return True

    for color in range(k):
        if is_valid(v, color, labels_assigned, G, k):
            labels_assigned[v] = color
            if tree_node_labelling_util(G, k, labels_assigned, v + 1):
                return True
            labels_assigned[v] = -1

    return False

def tree_node_labelling(G, k):
    labels_assigned = [-1] * len(G)
    if not tree_node_labelling_util(G, k, labels_assigned, 0):
        return None
    return labels_assigned


# Load input instances
trees = pickle.load(open("Examples_of_AdjLists_of_Trees", "rb"))
k_values = pickle.load(open("Examples_of_k_values", "rb"))

# Compute solutions
solutions = []

# Compute proximity ratio
max_ratios = []
index = 0
for i in range(len(trees)):
    G = trees[i]
    k = k_values[i]
    result = tree_node_labelling(G, k)
    solutions.append(list(result))

    r_values, m_values = r_m_values(G, result, k)
    ratios = [r_values[v] / m_values[v] for v in range(len(G))]
    max_ratios.append(max(ratios))
    print(index,'r value ', r_values)
    print(index, 'm value ', m_values)
    index = index + 1
    
# Save solutions
print('\n')
print(solutions)            
print(max_ratios)
print(max(max_ratios))

